<a href="https://colab.research.google.com/github/Jisang-hwang93/BOK_Report_Team1/blob/minutes_pre_processing/(%EA%B2%B0%EC%B8%A1%EC%B9%98%EC%88%98%EC%A0%95)news_n_gram_labeling_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from google.colab import drive
import datetime

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
data_1 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/자연어 처리 1조/데이터셋/news_handle.csv', index_col=[0], encoding = 'utf-8')

In [4]:
data_1

,new_date,token_ngram
0,2005-01-01,"['만기/NNG', '국채/NNG', '수익률/NNG', 'fed/NNG', 'fe..."
1,2005-01-01,"['만기/NNG', '국채/NNG', '가격/NNG', '최악/NNG', '보내/V..."
2,2005-01-01,"['마지막/NNG', '거래일/NNG', '뉴욕/NNG', '환시/NNG', '달러..."
3,2005-01-01,"['마지막/NNG', '날/NNG', '뉴욕/NNG', '금융시장/NNG', '한산..."
4,2005-01-02,"['주도/NNG', '연속/NNG', '대비/NNG', '상승/NNG', '하/XS..."
...,...,...
261812,2019-12-31,"['중/NNG', '금융기관/NNG', '금리/NNG', '주택담보대출/NNG', ..."
261813,2019-12-31,"['신년사/NNG', '기자/NNG', '가장/MAG', '주력/NNG', '하/V..."
261814,2019-12-31,"['보고서/NNG', '기자/NNG', '세계/NNG', '경제/NNG', '미/N..."
261815,2019-12-31,"['농협/NNG', '수협/NNG', '이동/NNG', '빈/VV', '행장/NNG..."


In [11]:
data_1['new_date'] = data_1['new_date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
data_1 = data_1[data_1['new_date'] <= datetime.datetime(2017, 12, 31, 0, 0, 0,)]
data_1

,new_date,token_ngram
0,2005-01-01,"['만기/NNG', '국채/NNG', '수익률/NNG', 'fed/NNG', 'fe..."
1,2005-01-01,"['만기/NNG', '국채/NNG', '가격/NNG', '최악/NNG', '보내/V..."
2,2005-01-01,"['마지막/NNG', '거래일/NNG', '뉴욕/NNG', '환시/NNG', '달러..."
3,2005-01-01,"['마지막/NNG', '날/NNG', '뉴욕/NNG', '금융시장/NNG', '한산..."
4,2005-01-02,"['주도/NNG', '연속/NNG', '대비/NNG', '상승/NNG', '하/XS..."
...,...,...
258450,2017-12-31,"['절상/NNG', '중/NNG', '실질가치/NNG', '오르/VV', '중/NN..."
258451,2017-12-31,"['자료/NNG', '일자/NNG', '최고/NNG', '금리/NNG', '박의/N..."
258452,2017-12-31,"['뉴욕/NNG', '은실/NNG', '특파원/NNG', '거래/NNG', '주/N..."
258453,2017-12-31,"['판매/NNG', '정기/NNG', '예금/NNG', '중/NNG', '금리/NN..."


### 콜 금리 가져오기

In [22]:
ir = pd.read_csv('/content/drive/My Drive/Colab Notebooks/자연어 처리 1조/데이터셋/interest_rate.csv', encoding='utf-8', index_col = [0])
ir['new_date'] = ir.date.apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
ir.drop(['date', '기준 금리'], axis=1, inplace=True)
ir[ir['콜 금리'].notna()]

,콜 금리,new_date
12,3.27,2004-12-09
13,3.28,2004-12-10
14,3.28,2004-12-13
15,3.28,2004-12-14
16,3.28,2004-12-15
...,...,...
3885,0.44,2020-07-20
3886,0.46,2020-07-21
3887,0.45,2020-07-22
3888,0.46,2020-07-23


In [23]:
full_date = pd.DataFrame(pd.date_range('2004-12-01','2017-12-31'), columns=['new_date'])
full_date

,new_date
0,2004-12-01
1,2004-12-02
2,2004-12-03
3,2004-12-04
4,2004-12-05
...,...
4774,2017-12-27
4775,2017-12-28
4776,2017-12-29
4777,2017-12-30


In [24]:
full_ir = pd.merge(full_date, ir, how='left', on='new_date')
full_ir['콜 금리'] = full_ir['콜 금리'].fillna(method = 'ffill') #결측치는 가장 가까운 이전일의 콜금리로 채운다
full_ir['콜 금리'] = full_ir['콜 금리'].fillna(method = 'bfill') #2004년 12월 9일 이전 날짜는 2004년 12월 9일의 금리로 채운다 
full_ir['before_month'] = full_ir['new_date']

full_ir


,new_date,콜 금리,before_month
0,2004-12-01,3.27,2004-12-01
1,2004-12-02,3.27,2004-12-02
2,2004-12-03,3.27,2004-12-03
3,2004-12-04,3.27,2004-12-04
4,2004-12-05,3.27,2004-12-05
...,...,...,...
4774,2017-12-27,1.51,2017-12-27
4775,2017-12-28,1.55,2017-12-28
4776,2017-12-29,1.58,2017-12-29
4777,2017-12-30,1.58,2017-12-30


In [25]:
# 1. new_date에 대해 콜금리 컬럼 생성
data = pd.merge(data_1, full_ir, how = 'left', on='new_date')
data

,new_date,token_ngram,콜 금리,before_month
0,2005-01-01,"['만기/NNG', '국채/NNG', '수익률/NNG', 'fed/NNG', 'fe...",3.29,2005-01-01
1,2005-01-01,"['만기/NNG', '국채/NNG', '가격/NNG', '최악/NNG', '보내/V...",3.29,2005-01-01
2,2005-01-01,"['마지막/NNG', '거래일/NNG', '뉴욕/NNG', '환시/NNG', '달러...",3.29,2005-01-01
3,2005-01-01,"['마지막/NNG', '날/NNG', '뉴욕/NNG', '금융시장/NNG', '한산...",3.29,2005-01-01
4,2005-01-02,"['주도/NNG', '연속/NNG', '대비/NNG', '상승/NNG', '하/XS...",3.29,2005-01-02
...,...,...,...,...
253022,2017-12-31,"['절상/NNG', '중/NNG', '실질가치/NNG', '오르/VV', '중/NN...",1.58,2017-12-31
253023,2017-12-31,"['자료/NNG', '일자/NNG', '최고/NNG', '금리/NNG', '박의/N...",1.58,2017-12-31
253024,2017-12-31,"['뉴욕/NNG', '은실/NNG', '특파원/NNG', '거래/NNG', '주/N...",1.58,2017-12-31
253025,2017-12-31,"['판매/NNG', '정기/NNG', '예금/NNG', '중/NNG', '금리/NN...",1.58,2017-12-31


In [26]:
# # 2. before_month에 대해 콜금리 컬럼 생성
data.before_month = data.new_date.apply(lambda x: x - datetime.timedelta(days=30))
data = pd.merge(data, full_ir, how = 'left', on='before_month')

# # 3. 콜금리 변화 컬럼(change) 생성
data['change'] = data['콜 금리_x'] - data['콜 금리_y']
data

,new_date_x,token_ngram,콜 금리_x,before_month,new_date_y,콜 금리_y,change
0,2005-01-01,"['만기/NNG', '국채/NNG', '수익률/NNG', 'fed/NNG', 'fe...",3.29,2004-12-02,2004-12-02,3.27,0.02
1,2005-01-01,"['만기/NNG', '국채/NNG', '가격/NNG', '최악/NNG', '보내/V...",3.29,2004-12-02,2004-12-02,3.27,0.02
2,2005-01-01,"['마지막/NNG', '거래일/NNG', '뉴욕/NNG', '환시/NNG', '달러...",3.29,2004-12-02,2004-12-02,3.27,0.02
3,2005-01-01,"['마지막/NNG', '날/NNG', '뉴욕/NNG', '금융시장/NNG', '한산...",3.29,2004-12-02,2004-12-02,3.27,0.02
4,2005-01-02,"['주도/NNG', '연속/NNG', '대비/NNG', '상승/NNG', '하/XS...",3.29,2004-12-03,2004-12-03,3.27,0.02
...,...,...,...,...,...,...,...
253022,2017-12-31,"['절상/NNG', '중/NNG', '실질가치/NNG', '오르/VV', '중/NN...",1.58,2017-12-01,2017-12-01,1.48,0.10
253023,2017-12-31,"['자료/NNG', '일자/NNG', '최고/NNG', '금리/NNG', '박의/N...",1.58,2017-12-01,2017-12-01,1.48,0.10
253024,2017-12-31,"['뉴욕/NNG', '은실/NNG', '특파원/NNG', '거래/NNG', '주/N...",1.58,2017-12-01,2017-12-01,1.48,0.10
253025,2017-12-31,"['판매/NNG', '정기/NNG', '예금/NNG', '중/NNG', '금리/NN...",1.58,2017-12-01,2017-12-01,1.48,0.10


In [27]:
# 콜금리 변화에 따른 라벨링

def labeling(change):
    if change > 0.03:
        return 1
    elif change < -0.03:
        return -1
    else:
        return 0
        
data['label'] = data.change.apply(lambda x: labeling(x))
data

,new_date_x,token_ngram,콜 금리_x,before_month,new_date_y,콜 금리_y,change,label
0,2005-01-01,"['만기/NNG', '국채/NNG', '수익률/NNG', 'fed/NNG', 'fe...",3.29,2004-12-02,2004-12-02,3.27,0.02,0
1,2005-01-01,"['만기/NNG', '국채/NNG', '가격/NNG', '최악/NNG', '보내/V...",3.29,2004-12-02,2004-12-02,3.27,0.02,0
2,2005-01-01,"['마지막/NNG', '거래일/NNG', '뉴욕/NNG', '환시/NNG', '달러...",3.29,2004-12-02,2004-12-02,3.27,0.02,0
3,2005-01-01,"['마지막/NNG', '날/NNG', '뉴욕/NNG', '금융시장/NNG', '한산...",3.29,2004-12-02,2004-12-02,3.27,0.02,0
4,2005-01-02,"['주도/NNG', '연속/NNG', '대비/NNG', '상승/NNG', '하/XS...",3.29,2004-12-03,2004-12-03,3.27,0.02,0
...,...,...,...,...,...,...,...,...
253022,2017-12-31,"['절상/NNG', '중/NNG', '실질가치/NNG', '오르/VV', '중/NN...",1.58,2017-12-01,2017-12-01,1.48,0.10,1
253023,2017-12-31,"['자료/NNG', '일자/NNG', '최고/NNG', '금리/NNG', '박의/N...",1.58,2017-12-01,2017-12-01,1.48,0.10,1
253024,2017-12-31,"['뉴욕/NNG', '은실/NNG', '특파원/NNG', '거래/NNG', '주/N...",1.58,2017-12-01,2017-12-01,1.48,0.10,1
253025,2017-12-31,"['판매/NNG', '정기/NNG', '예금/NNG', '중/NNG', '금리/NN...",1.58,2017-12-01,2017-12-01,1.48,0.10,1


In [28]:
data.drop(['new_date_x', '콜 금리_x', 'before_month', '콜 금리_y', 'new_date_y', 'change'], axis='columns', inplace=True)
data

,token_ngram,label
0,"['만기/NNG', '국채/NNG', '수익률/NNG', 'fed/NNG', 'fe...",0
1,"['만기/NNG', '국채/NNG', '가격/NNG', '최악/NNG', '보내/V...",0
2,"['마지막/NNG', '거래일/NNG', '뉴욕/NNG', '환시/NNG', '달러...",0
3,"['마지막/NNG', '날/NNG', '뉴욕/NNG', '금융시장/NNG', '한산...",0
4,"['주도/NNG', '연속/NNG', '대비/NNG', '상승/NNG', '하/XS...",0
...,...,...
253022,"['절상/NNG', '중/NNG', '실질가치/NNG', '오르/VV', '중/NN...",1
253023,"['자료/NNG', '일자/NNG', '최고/NNG', '금리/NNG', '박의/N...",1
253024,"['뉴욕/NNG', '은실/NNG', '특파원/NNG', '거래/NNG', '주/N...",1
253025,"['판매/NNG', '정기/NNG', '예금/NNG', '중/NNG', '금리/NN...",1


In [30]:
data.to_csv(r'/content/drive/My Drive/Colab Notebooks/자연어 처리 1조/데이터셋/news_handle_notna.csv')